In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import snscrape.modules.twitter as sntwitter
import sec_api

**Defining tickers**

In [53]:
ticker = "AAPL"

**Twitter scraping example**

In [ ]:
max_tweets = 100
all_tweets = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(ticker).get_items()):
    if i > max_tweets:
        break
    
    all_tweets.append(tweet.content)

print(all_tweets)